<a href="https://colab.research.google.com/github/ridhima31/VIZ-Test/blob/master/Windows_pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install PySpark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for PySpark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=eeb9bf60d1dd6f31b9a0a2192a4077199ee1faf596fed82742e1da0f826fdf9a
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built PySpark


In [2]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [101]:
df = spark.createDataFrame(
    [([["a","b","c"], ["d","e","f"], ["g","h","i", "j"]],)],
    ["data"]
)

In [102]:
df.show()

+--------------------+
|                data|
+--------------------+
|[[a, b, c], [d, e...|
+--------------------+



In [116]:
df1=df.withColumn("data1",F.explode("data"))

In [135]:
maxsize=df1.select(F.max(F.size("data1"))).collect()

In [136]:
maxsize[0][0]

4

In [138]:
df1.select(*[df1["data1"][i].alias(f"col:{i}") for i in range(maxsize[0][0])]).show()

+-----+-----+-----+-----+
|col:0|col:1|col:2|col:3|
+-----+-----+-----+-----+
|    a|    b|    c| NULL|
|    d|    e|    f| NULL|
|    g|    h|    i|    j|
+-----+-----+-----+-----+



# New problem to join 2 dataframes

In [139]:
from pyspark.sql import Window
from datetime import datetime

In [140]:
data = [("985XFT82Y4", "20200824"),
("985XFT82Y4", "20200826"),
("985XFT82Y4", "20200902"),
("985XFT82Y4", "20200918"),
("985XFT82Y4", "20200930"),
("985XFT82Y4", "20201016"),
("985XFT82Y4", "20201021"),
("985XFT82Y4", "20201102"),
("985XFT82Y4", "20201111"),
("985XFT82Y4", "20201112"),
("985XFT82Y4", "20201208"),
("985XFT82Y4", "20210111"),
("985XFT82Y4", "20210202"),
("985XFT82Y4", "20210303"),
("985XFT82Y4", "20210309"),
("985XFT82Y4", "20210311")]

In [142]:
p2_df1=spark.createDataFrame(data,["cust_id","pd_dt"]).withColumn("pd_dt",F.to_timestamp("pd_dt","yyyymmdd")).withColumn("pd_dt",F.date_format("pd_dt","dd/mm/yyyy"))

In [143]:
p2_df1.show()

+----------+----------+
|   cust_id|     pd_dt|
+----------+----------+
|985XFT82Y4|24/08/2020|
|985XFT82Y4|26/08/2020|
|985XFT82Y4|02/09/2020|
|985XFT82Y4|18/09/2020|
|985XFT82Y4|30/09/2020|
|985XFT82Y4|16/10/2020|
|985XFT82Y4|21/10/2020|
|985XFT82Y4|02/11/2020|
|985XFT82Y4|11/11/2020|
|985XFT82Y4|12/11/2020|
|985XFT82Y4|08/12/2020|
|985XFT82Y4|11/01/2021|
|985XFT82Y4|02/02/2021|
|985XFT82Y4|03/03/2021|
|985XFT82Y4|09/03/2021|
|985XFT82Y4|11/03/2021|
+----------+----------+



In [144]:
data1 = [("985XFT82Y4", "2020-08-24 22:12:34.332000",  "subscribed"),
("985XFT82Y4", "2020-11-11 14:45:31.152000",  "installed"),
("985XFT82Y4", "2021-02-02 01:26:34.500000",  "migration"),
("985XFT82Y4", "2021-03-09 08:11:57.790000",  "setup done")]

In [184]:
p2_df2=spark.createDataFrame(data1,["cust_id","chg_date","ins_status"]).withColumn("chg_date",F.to_timestamp("chg_date",'yyyy-MM-dd HH:mm:ss.SSSSSS')).withColumn("chg_date",F.date_format('chg_date','dd-MM-yyyy'))

In [185]:
p2_df2.show()

+----------+----------+----------+
|   cust_id|  chg_date|ins_status|
+----------+----------+----------+
|985XFT82Y4|24-08-2020|subscribed|
|985XFT82Y4|11-11-2020| installed|
|985XFT82Y4|02-02-2021| migration|
|985XFT82Y4|09-03-2021|setup done|
+----------+----------+----------+



In [186]:
wd=Window.partitionBy('cust_id').orderBy("chg_date")

In [197]:
p2_df3=p2_df2.withColumn("next_chnage", F.lead("chg_date").over(wd)).orderBy("chg_date")

In [210]:
cond=(p2_df3['cust_id']==p2_df1['cust_id']) & (p2_df1['pd_dt']>=p2_df3["chg_date"] )& (p2_df1['pd_dt']<p2_df3["next_chnage"])

In [215]:
p2_df1.join(p2_df3,cond).select(p2_df1["cust_id"],p2_df1["pd_dt"],p2_df3["ins_status"]).orderBy(p2_df1["pd_dt"]).show()

+----------+----------+----------+
|   cust_id|     pd_dt|ins_status|
+----------+----------+----------+
|985XFT82Y4|02/02/2021| migration|
|985XFT82Y4|02/09/2020| migration|
|985XFT82Y4|02/11/2020| migration|
|985XFT82Y4|03/03/2021| migration|
|985XFT82Y4|08/12/2020| migration|
|985XFT82Y4|09/03/2021|setup done|
|985XFT82Y4|11/01/2021| installed|
|985XFT82Y4|11/03/2021| installed|
|985XFT82Y4|11/11/2020| installed|
|985XFT82Y4|12/11/2020| installed|
|985XFT82Y4|16/10/2020| installed|
|985XFT82Y4|18/09/2020| installed|
|985XFT82Y4|21/10/2020| installed|
+----------+----------+----------+

